In [89]:
#imports and formatting
#libraries to import
import requests
import os
import pandas as pd
import json
import time
import numpy as np
import datetime
pd.set_option('display.max_columns', None)

In [90]:
#import dataframes 
#inspect dfs
#merge and clean duplicates, nulls etc 

In [91]:
#import API data and scraped data for US and Canadian markets
US_data = pd.read_csv("C:\\Users\\IRC-240\\Desktop\\LHL Bootcamo\\LHL_DS-Midterm-Project\\data\\top500_companies_US.csv")


In [92]:
#inspect dfs, check for nulls, check for dupes and drop
# check data types are proper and change if needed


Clean US data from Polygon API

In [93]:
#check for nulls for US Market Data 
US_data.isnull().sum()


Date                  2
Company Name          0
Ticker Symbol         0
Oct2023 Market Cap    0
Open                  2
Low                   2
High                  2
Close                 2
Volume                2
dtype: int64

In [94]:
#further inspect missing data
missing_USdata = US_data[US_data['Date'].isnull()] , [US_data['Open'].isnull()] , [US_data['Low'].isnull()] , [US_data['High'].isnull()] , [US_data['Close'].isnull()], [US_data['Volume'].isnull()]
missing_USdata

(       Date         Company Name Ticker Symbol Oct2023 Market Cap  Open  Low  \
 2922    NaN  Berkshire Hathaway          BRK-B          $744.39 B   NaN  NaN   
 132599  NaN         Brown Forman          BF-A           $27.13 B   NaN  NaN   
 
         High  Close  Volume  
 2922     NaN    NaN     NaN  
 132599   NaN    NaN     NaN  ,
 [0         False
  1         False
  2         False
  3         False
  4         False
            ...  
  247626    False
  247627    False
  247628    False
  247629    False
  247630    False
  Name: Open, Length: 247631, dtype: bool],
 [0         False
  1         False
  2         False
  3         False
  4         False
            ...  
  247626    False
  247627    False
  247628    False
  247629    False
  247630    False
  Name: Low, Length: 247631, dtype: bool],
 [0         False
  1         False
  2         False
  3         False
  4         False
            ...  
  247626    False
  247627    False
  247628    False
  247629    Fals

In [95]:
#get the surrounding data for the missing data to give context and help drive data cleaning decision to impute the data or remove
for index in [2922, 132599]:
    display(US_data.iloc[index-3:index+4])

,Date,Company Name,Ticker Symbol,Oct2023 Market Cap,Open,Low,High,Close,Volume
2919,2023-10-26,Meta Platforms (Facebook),META,$774.22 B,295.0000,279.4030,295.0000,288.3500,66684141.0
2920,2023-10-27,Meta Platforms (Facebook),META,$774.22 B,294.4800,292.9700,299.3100,296.7300,29596256.0
2921,2023-10-30,Meta Platforms (Facebook),META,$774.22 B,299.0900,299.0500,309.3967,302.6600,28435055.0
2922,NaN,Berkshire Hathaway,BRK-B,$744.39 B,NaN,NaN,NaN,NaN,NaN
2923,2021-11-01,Tesla,TSLA,$638.45 B,381.6667,372.8867,403.2500,402.8633,168128235.0
2924,2021-11-02,Tesla,TSLA,$638.45 B,386.4517,382.0000,402.8633,390.6667,128135646.0
2925,2021-11-03,Tesla,TSLA,$638.45 B,392.4433,384.2067,405.1300,404.6200,103877037.0


,Date,Company Name,Ticker Symbol,Oct2023 Market Cap,Open,Low,High,Close,Volume
132596,2023-10-26,GlobalFoundries,GFS,$27.42 B,51.54,51.5400,53.0700,52.06,1222979.0
132597,2023-10-27,GlobalFoundries,GFS,$27.42 B,52.41,51.7200,52.7400,51.98,702067.0
132598,2023-10-30,GlobalFoundries,GFS,$27.42 B,51.98,49.7100,52.0000,49.91,1640764.0
132599,NaN,Brown Forman,BF-A,$27.13 B,NaN,NaN,NaN,NaN,NaN
132600,2021-11-01,Cheniere Energy\r\n,CQP,$26.98 B,43.94,43.6450,44.5066,44.10,159016.0
132601,2021-11-02,Cheniere Energy\r\n,CQP,$26.98 B,44.23,42.7500,44.2300,43.71,195260.0
132602,2021-11-03,Cheniere Energy\r\n,CQP,$26.98 B,43.97,42.4301,43.9700,43.59,231537.0


In [96]:
#after inspection I realized that the missing dates for both stocks is 2023 Oct 31, this API data was pulled on that same date and may result in the NaN values, for the purpose of the analysis it is okay to remove it as it will not skew data, we will just have a slightly smaller data set to work with. 
#the missing data does not suggest continuity as that is that last date in the API call parameter and will not skew data if excluded
# remove the NaN rows, then reset index for proper continuity

US_data_cleaned = US_data.dropna(subset= ['Date', 'Open', 'Close', 'High', 'Low', 'Volume'])

US_data_cleaned.reset_index(drop=True, inplace=True)

#QA check                         
print(US_data_cleaned.isnull().sum())

#passed 

Date                  0
Company Name          0
Ticker Symbol         0
Oct2023 Market Cap    0
Open                  0
Low                   0
High                  0
Close                 0
Volume                0
dtype: int64


In [97]:
#Check for duplicate rows 
US_data_cleaned.duplicated().sum()

0

In [98]:
#verify and optimize data types for EDA
print(US_data_cleaned.dtypes)
US_data_cleaned.head()

Date                   object
Company Name           object
Ticker Symbol          object
Oct2023 Market Cap     object
Open                  float64
Low                   float64
High                  float64
Close                 float64
Volume                float64
dtype: object


,Date,Company Name,Ticker Symbol,Oct2023 Market Cap,Open,Low,High,Close,Volume
0,2021-11-01,Apple,AAPL,$2.669 T,148.985,147.80,149.70,148.96,74588257.0
1,2021-11-02,Apple,AAPL,$2.669 T,148.660,148.65,151.57,150.02,69121986.0
2,2021-11-03,Apple,AAPL,$2.669 T,150.390,149.82,151.97,151.49,54511532.0
3,2021-11-04,Apple,AAPL,$2.669 T,151.580,150.64,152.43,150.96,60394613.0
4,2021-11-05,Apple,AAPL,$2.669 T,151.890,150.06,152.20,151.28,65163883.0


In [99]:
#convert date to datetime data type, convert market cap to float dtype
US_data_cleaned['Date'] = pd.to_datetime(US_data_cleaned['Date'])


#to convert market cap i need to account for million billion and trillion valuations and convert accordingly 
def convert_market_cap(valuation):
    valuation = valuation.replace('$', '').replace(',', '')
    
    #check for Million or Billion or Trillion valuation and convert to float based on dollar value
    if 'M' in valuation:
        return float(valuation.replace('M','')) * 1e6 #multiply by digits in million 
    elif 'B' in  valuation:
        return float(valuation.replace('B','')) * 1e9 #multiple by digits in billion 
    elif 'T' in valuation:
        return float(valuation.replace('T','')) * 1e12 # multiply by digits in trillion 
    else:
        return float(valuation)

US_data_cleaned['Oct2023 Market Cap'] = US_data_cleaned['Oct2023 Market Cap'].apply(convert_market_cap)


#QA check 
US_data_cleaned.dtypes
#passed 


C:\Users\IRC-240\AppData\Local\Temp\ipykernel_41952\2918158057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_data_cleaned['Date'] = pd.to_datetime(US_data_cleaned['Date'])
C:\Users\IRC-240\AppData\Local\Temp\ipykernel_41952\2918158057.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_data_cleaned['Oct2023 Market Cap'] = US_data_cleaned['Oct2023 Market Cap'].apply(convert_market_cap)


Date                  datetime64[ns]
Company Name                  object
Ticker Symbol                 object
Oct2023 Market Cap           float64
Open                         float64
Low                          float64
High                         float64
Close                        float64
Volume                       float64
dtype: object

Clean Canadian Stock Market Data from Web Scraping

In [100]:
CAN_data =  pd.read_csv("C:\\Users\\IRC-240\\Desktop\\LHL Bootcamo\\LHL_DS-Midterm-Project\\data\\top500_companies_canada.csv")

In [101]:
# first sort the df by highest market cap, then only use the top 100 stocks, then check nulls,dupes, outliers, and dtypes 
# DF should consist of the top 100 stocks from the Canadian Markets

In [102]:
#convert dtypes market cap to float and date to date time 
CAN_data['Date'] = pd.to_datetime(US_data_cleaned['Date'])

#convert market cap to float64 using function made ealier 
CAN_data['Oct2023 Market Cap'] = CAN_data['Oct2023 Market Cap'].apply(convert_market_cap)


In [104]:
#QA check the data 
CAN_data.dtypes
#passed


Date                  datetime64[ns]
Company Name                  object
Ticker Symbol                 object
Oct2023 Market Cap           float64
Open                         float64
Low                          float64
High                         float64
Close                        float64
Volume                       float64
dtype: object

In [118]:
#sort df by highest marketcap 
CAN_data_sorted = CAN_data.sort_values(by='Oct2023 Market Cap', ascending=False)
#group by compay name and market cap value 
top100 = CAN_data_sorted.groupby('Company Name')['Oct2023 Market Cap'].max().sort_values(ascending=False).head(100)

#parse original dataframe and filter out the companies that arent in the top 100 

top100_can_data = CAN_data_sorted[CAN_data_sorted['Company Name'].isin(top100.index)]


In [119]:
#QA check 
top100_can_data['Company Name'].nunique()
#passed 

100

In [125]:
# check for nulls 
top100_can_data.isnull().sum()



,Date,Company Name,Ticker Symbol,Oct2023 Market Cap,Open,Low,High,Close,Volume
4518,2022-03-11,Alimentation Couche-Tard\r\n,ATD.TO,5.254000e+10,NaN,NaN,NaN,NaN,NaN
5745,2023-01-31,Constellation Software\r\n,CSU.TO,4.244000e+10,NaN,NaN,NaN,NaN,NaN
11268,2023-02-01,Loblaw Companies,L.TO,2.585000e+10,NaN,NaN,NaN,NaN,NaN
11269,2023-02-02,Great-West Lifeco\r\n,GWO.TO,2.577000e+10,NaN,NaN,NaN,NaN,NaN
11270,2023-02-03,Intact Financial,IFC.TO,2.503000e+10,NaN,NaN,NaN,NaN,NaN
13279,2023-02-06,Fairfax Financial\r\n,FFH.TO,2.153000e+10,NaN,NaN,NaN,NaN,NaN
13280,2023-02-07,National Bank of Canada\r\n,NA.TO,2.101000e+10,NaN,NaN,NaN,NaN,NaN
14285,2023-02-08,Dollarama,DOL.TO,1.928000e+10,NaN,NaN,NaN,NaN,NaN
15290,2023-02-09,Tourmaline Oil\r\n,TOU.TO,1.795000e+10,NaN,NaN,NaN,NaN,NaN
16295,2023-02-10,WSP Global\r\n,WSP.TO,1.628000e+10,NaN,NaN,NaN,NaN,NaN


In [ ]:
top100_can_data[top100_can_data.isnull().any(axis=1)]
